## 抓取issues相關資訊 (將資料存成quests.json 以進一步做成任務列表)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json

driver = webdriver.Firefox()
driver.get("https://github.com/Astrohackers-TW/IANCUPythonAdventure/issues") 
issues = driver.find_elements_by_class_name("Box-row-link") 

issue_title = []
issue_url = []
issue_description = []
for issue in issues:
    issue_title.append(issue.text)
    issue_url.append(issue.get_attribute('href'))

for url in issue_url:
    driver.get(url)
    description = driver.find_element_by_css_selector(".timeline-comment-header+.edit-comment-hide").text
    issue_description.append(description)

driver.close()

quests = [{'title': title, 'description': description} for title, description in zip(issue_title, issue_description)]
with open('quests.json', 'w') as f:
    f.write(json.dumps(quests, ensure_ascii=False, indent=1))

## 抓取GitHub帳號的大頭貼網址

In [2]:
from selenium import webdriver
import urllib.request
import json

with open('adventurers_template.json', 'r') as f:
    dic_adventurers = json.load(f)

driver = webdriver.Firefox()

# 更新每個adventurer的avatar
for adventurer in list(dic_adventurers.keys()):
    driver.get("https://github.com/" + adventurer)
    avatar = driver.find_element_by_class_name("avatar").get_attribute('src') 
    dic_adventurers[adventurer]['avatar'] = avatar

driver.close()

with open('adventurers.json', 'w') as f:
    f.write(json.dumps(dic_adventurers, ensure_ascii=False, indent=1, sort_keys=True))

## 抓取GitHub專案contributors的提交數量 (作為等級依據)

In [3]:
import numpy as np
# 更新每個adventurer的level
driver = webdriver.Firefox()
driver.get("https://github.com/Astrohackers-TW/IANCUPythonAdventure/graphs/contributors") 

adventurers_name = list(dic_adventurers.keys())
contributors = {}
names = driver.find_elements_by_class_name("aname") 
commits = driver.find_elements_by_css_selector(".cmeta")
driver.close()

for n, c in zip(names, commits):
    count = int(c.text.split()[0])
    if count > 1:
        level = int(np.log10(count) / np.log10(2))
        this_level_count = 2 ** level
    else:
        level = 1;
        this_level_count = 0 
    next_level_count = 2 ** (level + 1)
    count_diff = next_level_count - this_level_count
    progress = next_level_count - count
    progress_percent = 100 * (count_diff - progress) / count_diff
    contributors.update({n.text: [str(count), str(level), str(progress), str(progress_percent)]})

contributors_name = list(contributors.keys())    
for name in adventurers_name:
    if name in contributors_name:
        dic_adventurers[name]['commits'] = contributors[name][0]
        dic_adventurers[name]['level'] = contributors[name][1]
        dic_adventurers[name]['progress'] = contributors[name][2]
        dic_adventurers[name]['progress_percent'] = contributors[name][3]
    
with open('adventurers.json', 'w') as f:
    f.write(json.dumps(dic_adventurers, ensure_ascii=False, indent=1, sort_keys=True))